<a href="https://colab.research.google.com/github/anubhavgupta1/Udacity/blob/main/Frameworks/Tensorflow/RNNs/Char%20RNN/shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


!mkdir -p drive
!google-drive-ocamlfuse drive

import os
os.chdir("drive/Udacity/RNN/Tensorflow Shakespear/")
!ls

sample_data
E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 146364 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.24-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0

## Import TensorFlow and other libraries

In [2]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

## Download the Shakespeare dataset

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


## Read the data

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))
# Take a look at the first 250 characters in text
print(text[:250])

Length of text: 1115394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

65 unique characters


# Process the text

## Vectorize the text

In [6]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
print(chars)

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>


In [7]:
#Now create the preprocessing.StringLookup layer
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab))

In [8]:
#It converts form tokens to character IDs, padding with 0:
ids = ids_from_chars(chars)
print(ids)

<tf.RaggedTensor [[41, 42, 43, 44, 45, 46, 47], [64, 65, 66]]>


In [9]:
print(ids_from_chars.get_vocabulary())

['', '[UNK]', '\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [10]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup( vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [11]:
chars = chars_from_ids(ids)
print(chars)

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>


In [12]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

## Create training examples and targets

In [14]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
print(all_ids)

tf.Tensor([20 49 58 ... 47 10  2], shape=(1115394,), dtype=int64)


In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
print(ids_dataset)

<TensorSliceDataset shapes: (), types: tf.int64>


In [16]:
for ids in ids_dataset.take(13):
    chars_from_ids_ = chars_from_ids(ids)
    print(chars_from_ids_.numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i
z
e
n


In [17]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
print(examples_per_epoch)

11043


In [18]:
#The batch method lets you easily convert these individual characters to sequences of the desired size
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print("Sequence is : ")
  print(seq)
  print("\nCharacter in sequence are : ")
  print(chars_from_ids(seq))
  print("\nText in sequence is : ")
  print(text_from_ids(seq).numpy())

Sequence is : 
tf.Tensor(
[20 49 58 59 60  3 17 49 60 49 66 45 54 12  2 16 45 46 55 58 45  3 63 45
  3 56 58 55 43 45 45 44  3 41 54 65  3 46 61 58 60 48 45 58  8  3 48 45
 41 58  3 53 45  3 59 56 45 41 51 10  2  2 15 52 52 12  2 33 56 45 41 51
  8  3 59 56 45 41 51 10  2  2 20 49 58 59 60  3 17 49 60 49 66 45 54 12
  2 39 55 61  3], shape=(101,), dtype=int64)

Character in sequence are : 
tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)

Text in sequence is : 
b'First Citizen:\nBefo

In [19]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [20]:
for seq in sequences.take(1):
  print("\nText in sequence is : ")
  _text_ = text_from_ids(seq).numpy()
  print(_text_)
  _input_text_, _target_text_ = split_input_target(_text_)
  print("\nInput Text is : ")
  print(_input_text_)
  print("\nTarget Text is : ")
  print(_target_text_)


Text in sequence is : 
b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '

Input Text is : 
b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

Target Text is : 
b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in  dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


## Create training batches

In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

print(dataset)

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


## Build The Model

In [24]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else: 
      return x

In [26]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

## Try the model

In [27]:
for input_example_batch, target_example_batch in dataset.take(1):
    print(input_example_batch.shape, target_example_batch.shape)

(64, 100) (64, 100)


In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 67) # (batch_size, sequence_length, vocab_size)


In [29]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  17152     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  68675     
Total params: 4,024,131
Trainable params: 4,024,131
Non-trainable params: 0
_________________________________________________________________


In [30]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
print(sampled_indices)

[53 18 15  9 17 53 22 60  9 53 66  7 59 50  3 66 48 39 50 37 29 33 52 33
 10 51 49  9 39 20 52 47 19 47 54 39 39 11 31 50  6 49 55 55 22  7 27  7
 52 44 31 48 47 55  2 26 18 36 29 57 15 46 39 29 49 50 64  6 56  3 39 20
 55 18 26 47 34 54 20  1 35  6 63 27 25 32 49 64 64  8 59 63 43 24 62  9
 19 29 53 29]


In [31]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ethought,--I\ncannot tell how to term it.\n\nFirst Servingman:\nHe had so; looking as it were--would I w'

Next Char Predictions:
 b"mDA-CmHt-mz'sj zhYjWOSlS.ki-YFlgEgnYY3Qj&iooH'M'ldQhgo\nLDVOqAfYOijx&p YFoDLgTnF[UNK]U&wMKRixx,swcJv-EOmO"


## Train the model

In [32]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [33]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 67)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.2042427


In [34]:
tf.exp(mean_loss).numpy()

66.969864

In [35]:
#Configure the training procedure using the tf.keras.Model.compile method. 
#Use tf.keras.optimizers.
#Adam with default arguments and the loss function.
model.compile(optimizer='adam', loss=loss)

## Configure checkpoints

In [36]:
#Use a tf.keras.callbacks.ModelCheckpoint 
#to ensure that checkpoints are saved during training:

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## Execute the training

In [37]:
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 9s 41ms/step - loss: 3.2877
Epoch 2/20
172/172 [==============================] - 8s 40ms/step - loss: 2.0798
Epoch 3/20
172/172 [==============================] - 8s 40ms/step - loss: 1.7621
Epoch 4/20
172/172 [==============================] - 8s 40ms/step - loss: 1.5769
Epoch 5/20
172/172 [==============================] - 8s 41ms/step - loss: 1.4649
Epoch 6/20
172/172 [==============================] - 8s 41ms/step - loss: 1.3861
Epoch 7/20
172/172 [==============================] - 8s 40ms/step - loss: 1.3305
Epoch 8/20
172/172 [==============================] - 8s 41ms/step - loss: 1.2824
Epoch 9/20
172/172 [==============================] - 8s 41ms/step - loss: 1.2406
Epoch 10/20
172/172 [==============================] - 8s 41ms/step - loss: 1.1954
Epoch 11/20
172/172 [==============================] - 8s 41ms/step - loss: 1.1560
Epoch 12/20
172/172 [==============================] - 8s 41ms/step - loss: 1.1113
Epoch 13/20
1

## Generate text

In [38]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())]) 
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits] 
    predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [39]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [40]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

print(f"\nRun time: {end - start}")

ROMEO:
She is my business; but I'll go raze very little elpe,
Upon his purple on thy body and her grave,
To brief and told me hath true here doth here alone
To let him and with a heart. Who's there a
curse is ready; mother, are all fall together:
That they this country's force smell of some
person, whose haughty man into his.

FLORIZEL:
Dear, and beggary fair one
gentleman that which you say so like a treecher.
Coroerinant Clarence, both brought your valour; I will.
Where dare discomes my prayers with the misther lites
Twice drawn overba-store?

ELBOW:
He may, let's about it. And tell us thither.

KING RICHARD III:
But lords, we have ended me to closent to follow me.

GLOUCESTER:

KING EDWARD IV:
This cast, you tell me, yea, and you, my God for what said
He?

TRANIO:
Signior Baptista! do you this alliance
Will slay thy matter to him, and I'll
From the Went-onime,
Were not besore. Your knees go: yet I do Nor
come down, good citizens: you know none, how licking,
As little counsel, let it

In [41]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result, '\n\n' + '_'*80)


print(f"\nRun time: {end - start}")

tf.Tensor(
[b"ROMEO:\nHear and ha! let her be kept on him:\nCall home, his nor tongues, travelling time\nThan have found scaved by--adised eyes,\nTo hurry for trimune from the warlike posture.\n\nPETRUCHIO:\nNoble lord!\n\nEDIOLANUS:\nDo so; anon, if you lord desire yourself\nHe of yourself to come abund again.\n\nKING EDWARD IV:\nWhy, then thou shalt become this fround intent,\nBy Bauthop's tap toicora, that are mark for them.\n\nKING EDWARD IV:\nThe cleager of mine honour, his noble issue\nA toich of micheries that makes me here.\nThat should by conscience takes upon each other.\nAnd let him spake this tempoxery to streak the back\nand her whose remedy: scroop about, even so\nAnd datch out with the bitter\nUpon his son shall bear a common\nOf my fortune to your affrage and bring\nWith treech and my repited to her.\n\nEXETES:\nAccursed, here I up stay, order thy life,\nI would deny her a strelt through 's thus;\nto the labelts know, which way, this deed dot I have heard\nThe scaful se

## Export the generator

In [42]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [43]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Go thank thee, Kate, I knew your hands and take it for.
I am on earth: yet I will teach a husband;



In [45]:
states = None
next_char = tf.constant(['We are accounted poor citizens'])
result = [next_char]

for n in range(1009):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

We are accounted poor citizens.

JOLINA:
Beseech you,--

First Citizen:
By prove itself: it best thou wrot thy lord, like an innocent million
Troops of love and lookness to come upon me. O, a month
More witchronce to conqueror and bow.
We dare ere you to? but I do not know
I mean, to do this fool.

GREMIO:
And promise the English corse is mere upon this,
For certain word: your sorrow's rest I live,
I would be cured by stem my out-to poison.

KING EDWARD IV:
Ay, Warwick, with a villain, what is yond frilar,
Avoided meat to beat both the banishmy of spirit
Unto the mind of Montague.

HENRY BOLINGBROKE:
This chartel to your does.

CORIOLANUS:
Have I none? where gust him please,
By charged Supl, God grant the bendned--

THOMPOLINGBROKE:
If it I was that elected thee!
Jest ambitious Edward, peace by flatter
There to a molehill, and all things changed for;
If it be sponet high fortune and the beauty
That, not believe to reliver.

PETRUCHIO:
You say it is no more. A' might I see my care
To ha